# Import dependencies

In [1]:
import cv2
import fnmatch
import matplotlib.pyplot
from os import listdir
from os.path import join
import numpy as np

# Set parameters

In [2]:
CAMERA_COUNT = 3
IMAGE_CENTER_REGULAR_EXPRESSION = 'center*'
PATH = './data'

file_center = 'IMG/center_2016_12_01_13_37_16_570.jpg'
file_left = 'IMG/left_2016_12_01_13_37_16_570.jpg'
file_right = 'IMG/right_2016_12_01_13_37_16_570.jpg'

# Get data

Use data by Udacity.

Source: https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip

Size: 322.8 MB

In [3]:
# Do not execute this cell as it will read all images into memory
# data = np.array([cv2.imread(join(PATH, file)) for file in listdir(PATH)], dtype = 'float32')

# Explore data

In [4]:
count = len(listdir(PATH))

print('Count:', count)
print('Frames:', count / CAMERA_COUNT)

Count: 2
Frames: 0.6666666666666666


In [5]:
data = np.array([1 for file in listdir(PATH) if fnmatch.fnmatch(file, IMAGE_CENTER_REGULAR_EXPRESSION)], dtype = 'float32')
print('Center image count:', data.sum())

Center image count: 0.0


In [6]:
image_center = cv2.imread(join(PATH, file_center))
print('Shape:', image_center.shape)

Shape: (160, 320, 3)


In [7]:
pyplot.imshow(cv2.cvtColor(image_center, cv2.COLOR_BGR2RGB))
pyplot.show()

NameError: name 'pyplot' is not defined

In [ ]:
image_left = cv2.imread(join(PATH, file_left))
pyplot.imshow(cv2.cvtColor(image_left, cv2.COLOR_BGR2RGB))
pyplot.show()

In [ ]:
image_right = cv2.imread(join(PATH, file_right))
pyplot.imshow(cv2.cvtColor(image_right, cv2.COLOR_BGR2RGB))
pyplot.show()

# Transform data

In [ ]:
image_center_resized = cv2.resize(image_center, (32, 16))
print('Shape:', image_center_resized.shape)

In [ ]:
pyplot.imshow(cv2.cvtColor(image_center_resized, cv2.COLOR_BGR2RGB))
pyplot.show()

In [ ]:
y_start = 60
y_end = image_center.shape[0] - 20
x_start = 0
x_end = image_center.shape[1]

image_center_cropped = image_center[y_start:y_end, x_start:x_end]
print('Shape:', image_center_cropped.shape)

In [ ]:
pyplot.imshow(cv2.cvtColor(image_center_cropped, cv2.COLOR_BGR2RGB))
pyplot.show()

In [ ]:
image_center_grayscale = cv2.cvtColor(image_center, cv2.COLOR_BGR2GRAY)

In [ ]:
pyplot.imshow(image_center_grayscale, cmap = 'gray')
pyplot.show()

In [ ]:
image_center_cropped_grayscale = cv2.cvtColor(image_center_cropped, cv2.COLOR_BGR2GRAY)
image_center_cropped_grayscale_resized = cv2.resize(image_center_cropped_grayscale, (32, 8))

In [ ]:
pyplot.imshow(image_center_cropped_grayscale_resized, cmap = 'gray')
pyplot.show()

# Get training set

# Get validation set

# Design model

# Train model

# Save model

# Load model

# Evaluate model

# Test model